In [39]:
import os 
import time
import random
import csv
import pandas as pd

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from datasets import load_dataset
from toxic_comment_collection import *

#get_dataset('basile2019')    
get_dataset('basile2019')


In [40]:
pd.options.mode.chained_assignment = None
basile_ = pd.read_csv("./files/basile2019/basile2019en.csv", sep="\t")#.to_numpy()
gibert_ = pd.read_csv("./files/gibert2018/gibert2018en.csv", sep="\t")#.to_numpy()
chung_ = pd.read_csv("./files/chung2019/chung2019.csv", sep="\t")#.to_numpy()
#print(chung_[0,1])
#print(gibert_[0,1])
del basile_['id']
del gibert_['id']
del chung_['id']

basile_['labels'] = basile_['labels'].astype('|S40')
basile_['labels'].loc[basile_['labels'] == b'[]'] = 0
basile_['labels'].loc[basile_['labels'] != b'[]'] = 1
basile_data = basile_['text'].to_list()
basile_labels = basile_['labels'].to_list()
print(len(basile_data))
print(len(basile_labels))

10000
10000


In [55]:
gibert_['labels'] = gibert_['labels'].astype('|S40')
gibert_.drop(gibert_.index[gibert_['labels'] == b"['idk/skip']"], inplace=True)
gibert_.drop(gibert_.index[gibert_['labels'] == b"['relation']"], inplace=True)
gibert_['labels'].loc[gibert_['labels'] == b"['none']"] = 0
gibert_['labels'].loc[gibert_['labels'] == b"['hate']"] = 1
gibert_['labels'].loc[gibert_['labels'] == b'0'] = 0
gibert_['labels'].loc[gibert_['labels'] == b'1'] = 1
gibert_data = gibert_['text'].to_list()
gibert_labels = gibert_['labels'].to_list()
print(gibert_['labels'][5000])
#print(len(gibert_labels[gibert_labels['labels'] == 1]))


chung_['labels'] = chung_['labels'].astype('|S40')
chung_['labels'].loc[chung_['labels'] == b"['hate', 'Islamophobia']"] = 1
chung_['labels'].loc[chung_['labels'] == b'0'] = 0
chung_['labels'].loc[chung_['labels'] == b'1'] = 1
chung_data = chung_['text'].to_list()
chung_labels = chung_['labels'].to_list()
#print(chung_['text'][3])
#print(len(chung_labels))

qian_ = pd.read_csv("./files/qian2019/qian2019en_reddit.csv", sep="\t")
del qian_['id']
qian_['labels'] = qian_['labels'].astype('|S40')
qian_.drop(qian_.index[qian_['labels'] == b'[]'], inplace=True)
qian_['labels'].loc[qian_['labels'] == b"['hate']"] = 1
qian_data = qian_['text'].to_list()
qian_labels = qian_['labels'].to_list()
#print(qian_['labels'][18])


0


In [34]:
res_data = basile_data + chung_data + gibert_data + qian_data
print(len(res_data))
res_labels = basile_labels + chung_labels + gibert_labels + qian_labels
print(len(res_labels))

res = basile_.append([qian_, gibert_, chung_])
print(res)

import pickle
with open("hate_dataset", "wb") as fp:   #Pickling
   pickle.dump(res_data, fp)
 
with open("hate_labels", "wb") as fp:   
    pickle.dump(res_labels, fp)
    
with open("hate_total", "wb") as fp:   
    pickle.dump(res_labels, fp)

26815
26815
                                                  text labels
0    Hurray, saving us $$$ in so many ways @potus @...      1
1    Why would young fighting age men be the vast m...      1
2    @KamalaHarris Illegals Dump their Kids at the ...      1
3    NY Times: 'Nearly All White' States Pose 'an A...      1
4    Orban in Brussels: European leaders are ignori...      1
..                                                 ...    ...
851  These Bedouins treat women like beasts and cut...      1
852  Una donna che accetta di vivere nascosta sotto...      1
853  Una donna che permette di farsi coprire con un...      1
854  In Italia non possiamo accogliere donne che de...      1
855  A woman who accepts to live hidden under a vei...      1

[26815 rows x 2 columns]


In [37]:
res.to_csv('file_name.csv', index=False)
res.head()

,text,labels
0,"Hurray, saving us $$$ in so many ways @potus @...",1
1,Why would young fighting age men be the vast m...,1
2,@KamalaHarris Illegals Dump their Kids at the ...,1
3,NY Times: 'Nearly All White' States Pose 'an A...,1
4,Orban in Brussels: European leaders are ignori...,1
